In [1]:
#Installing necessary dependences:
#!pip install -q keras-nlp

In [2]:
#!pip list

In [3]:
#imports:
import tensorflow as tf
import tensorflow_text as tf_text
import keras
import pandas as pd
import numpy as np
import os
import re
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Dense, MultiHeadAttention, Softmax, TextVectorization, LayerNormalization
from keras import Model, Input
from keras.models import Sequential
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab

In [4]:
import keras_nlp

Using TensorFlow backend


In [5]:
tf.compat.v1.disable_eager_execution()

In [6]:
#Preprocessing Data
filepath = os.path.dirname(os.path.dirname(os.path.abspath("."))) + "/Datasets"
filepath = re.sub(r'\\', '/', filepath)
print(filepath)
X1 = pd.read_csv(filepath + "/Tatobea Project" + '/'+ "spa" +'.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:"English", 1:"Translated"})
X1 = X1[X1.notnull().all(axis = 1)]

C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets


In [7]:
print(X1)

                                                  English  \
0                                                     Go.   
1                                                     Go.   
2                                                     Go.   
3                                                     Go.   
4                                                     Hi.   
...                                                   ...   
139008  A carbon footprint is the amount of carbon dio...   
139009  Since there are usually multiple websites on a...   
139010  If you want to sound like a native speaker, yo...   
139011  It may be impossible to get a completely error...   
139012  One day, I woke up to find that God had put ha...   

                                               Translated  
0                                                     Ve.  
1                                                   Vete.  
2                                                   Vaya.  
3                          

In [8]:
X1['Translated'] = "<ES> " + X1['Translated']

In [9]:
print(X1)

                                                  English  \
0                                                     Go.   
1                                                     Go.   
2                                                     Go.   
3                                                     Go.   
4                                                     Hi.   
...                                                   ...   
139008  A carbon footprint is the amount of carbon dio...   
139009  Since there are usually multiple websites on a...   
139010  If you want to sound like a native speaker, yo...   
139011  It may be impossible to get a completely error...   
139012  One day, I woke up to find that God had put ha...   

                                               Translated  
0                                                <ES> Ve.  
1                                              <ES> Vete.  
2                                              <ES> Vaya.  
3                          

In [10]:
train_x = pd.DataFrame()

In [11]:
for d in [x[0] for x in os.walk(filepath + "/DGT-TM")]:
    print(d)
    print(d[-6:])
    #Ignores TMX files
    if d[-1] == ")" or d[-6:] == "DGT-TM":
        continue
    #This code actually works, however due to the dataset it takes 1+ hour to run.
    #Because of this, at least for now I'll be restricting it to just the EN-ES folder.
    te = True
    if d[-5:] != "EN-ES":
        te = False
        continue
    
    fp = re.sub(r'\\', '/', d)
    E = fp.split("/")[-1].split("-")[0]
    print(E)
    T = fp.split("/")[-1].split("-")[1]
    print(T)
    print(d + '/'+ E + "-" + T +'.txt')
    
    temp = pd.read_csv(d + '/'+ E + "-" + T +'.txt', sep='\t', header = None)[[0,1]].rename(columns = {0:"English", 1:"Translated"})
    temp = temp[temp.notnull().all(axis = 1)]
    temp['Translated'] = "<" + T + "> " + temp['Translated']
    
    print(temp.head())
    if te:
        X1 = pd.concat([X1, temp])
    else:
        train_x = pd.concat([train_x, temp])


C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM
DGT-TM
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN -NL
EN -NL
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-BG
\EN-BG
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-CS
\EN-CS
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-DA
\EN-DA
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-DE
\EN-DE
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-EL
\EN-EL
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-ES
\EN-ES
EN
ES
C:/Users/nickg/OneDrive/Documents/DAAN 570/Group Project/Datasets/DGT-TM\EN-ES/EN-ES.txt
                                             English  \
0            Commission Regulation (EC) No 1788/2004   
1                                 of 15 October 2004   
2  fixing the minimum selling prices for butter f...  

In [12]:
Y1 = pd.DataFrame()
Y1['Translated'] = X1.pop("Translated")


In [13]:
print(Y1)

                                                Translated
0                                                 <ES> Ve.
1                                               <ES> Vete.
2                                               <ES> Vaya.
3                                             <ES> Váyase.
4                                               <ES> Hola.
...                                                    ...
5696845                       <ES> Código país tercero [1]
5696846                   <ES> Valor global de importación
5696847  <ES> Nomenclatura de países fijada por el Regl...
5696848   <ES> El código «999» significa «otros orígenes».
5696849  <ES> DO L 337 de 24.12.1994, p. 66; Reglamento...

[5835639 rows x 1 columns]


In [14]:
train_x1, test_x, train_y1, test_y = train_test_split(X1, Y1, test_size=0.5, random_state=42)

In [15]:
try:
    train_y = train_x.pop("Translated")
except:
    #train_x doesn't have any portions, so we'll just describe it as a normal Dataframe
    train_y = pd.DataFrame()

In [16]:
train_x = pd.concat([train_x, train_x1])

In [17]:
train_y = pd.concat([train_y, train_y1])

In [18]:
del train_x1
del train_y1
del X1
del Y1
del temp

In [19]:
print(train_x)
print(train_y)
print(test_x)
print(test_y)

                                                   English
4569925  Repair and maintenance services of other profe...
5159865  If the policy-holder belongs to a group of und...
1200393                       Cash flow (% of total sales)
3355922                                 White mineral oils
3934638  Directive 2004/17/EC of the European Parliamen...
...                                                    ...
1431032                                         Division D
2095551                 Done at Brussels, 7 December 2012.
4787669  The project's strategy is grounded in SEESAC's...
4165720  In S.11.01 the 15th paragraph, second bullet o...
1553774                                           Whereas:

[2917819 rows x 1 columns]
                                                Translated
4569925  <ES> Servicios de reparación y mantenimiento d...
5159865  <ES> Si el tomador del seguro formara parte de...
1200393       <ES> Flujo de caja (% de las ventas totales)
3355922                     

In [20]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(2917819, 1)
(2917819, 1)
(2917820, 1)
(2917820, 1)


In [21]:
#train_x = train_x.astype(str)
#test_x = test_x.astype(str)

In [22]:
#X = pd.concat([train_x, test_x])
#Y = pd.concat([train_y, test_y])
#Y_SP = Y[Y['Translated'].str.contains("<ES>")]

In [23]:
#max_padding_series_org = X['English'].str.split(" ")
#max_padding_len_org = max_padding_series_org.str.len()
#print(max_padding_len_org)
#max_padding_size_org = max_padding_len_org.max() + 2

In [24]:
#The +2 is necessary because of the [START] and [END] tokens
#print(max_padding_size_org)

In [25]:
#max_padding_series_tar = Y_SP['Translated'].str.split(" ")
#max_padding_len_tar = max_padding_series_tar.str.len()
#print(max_padding_len_tar)
#max_padding_size_tar = max_padding_len_tar.max() + 2

In [26]:
#print(max_padding_size_tar)

In [27]:
#Code used to find the length of 
#vectorizer_org = CountVectorizer()
#count_org = vectorizer_org.fit_transform(X['English'])
#vectorizer_tar = CountVectorizer()
#count_tar = vectorizer_tar.fit_transform(Y_SP['Translated'])
#print(count_org, count_tar)

In [28]:
#print(count_org.shape)

In [29]:
#print(len(vectorizer_org.get_feature_names_out()))

In [30]:
#print(len(vectorizer_tar.get_feature_names_out()))

In [31]:
#The total number of words in the English corpus seems to be 285636.
#The total number of words in the Spanish corpus seems to be 340903.
org_vocab_length = 285636
tar_vocab_length = 340903

In [32]:
#Need to tokenize train_x, test_x, train_y, test_y
#This code creates the vocabulary and saves it to a text file.
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_org_args = dict(
    # The target vocabulary size
    vocab_size = org_vocab_length,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

bert_vocab_tar_args = dict(
    # The target vocabulary size
    vocab_size = tar_vocab_length,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [33]:
print(bert_tokenizer_params)


{'lower_case': True}


In [34]:
#X_tensor = tf.convert_to_tensor(X['English'])

In [35]:
#print(X_tensor.shape)

In [36]:
#X_dataset = tf.data.Dataset.from_tensor_slices(X_tensor)

In [37]:
#en_vocab = bert_vocab.bert_vocab_from_dataset(
#    X_dataset.batch(1000).prefetch(2),
#    **bert_vocab_org_args
#)

In [38]:
#with open("engvocab.txt", 'w', encoding="utf-8") as f:
#    for token in en_vocab:
#      print(token, file=f)

In [39]:
#Y_SP_tensor = tf.convert_to_tensor(Y_SP['Translated'])

In [40]:
#Y_SP_dataset = tf.data.Dataset.from_tensor_slices(Y_SP_tensor)

In [41]:
#es_vocab = bert_vocab.bert_vocab_from_dataset(
#    Y_SP_dataset.batch(1000).prefetch(2),
#    **bert_vocab_tar_args
#)

In [42]:
#with open("esvocab.txt", 'w', encoding="utf-8") as f:
#    for token in es_vocab:
#      print(token, file=f)

In [43]:
file = open('engvocab.txt', 'r', encoding="utf-8")
input_vocab = len(file.read().split("\n"))
file.close()

In [44]:
file = open('esvocab.txt', 'r', encoding="utf-8")
output_vocab = len(file.read().split("\n"))
file.close()

In [45]:
print(input_vocab, output_vocab)

86320 100347


In [46]:
es_tokenizer = tf_text.BertTokenizer('esvocab.txt', **bert_tokenizer_params)
en_tokenizer = tf_text.BertTokenizer('engvocab.txt', **bert_tokenizer_params)

In [47]:
import gc
gc.collect()
max_length = 256

In [48]:
x_train_tensor = en_tokenizer.tokenize(train_x['English'])
x_train_tensor = x_train_tensor.merge_dims(-2,-1)
#Max length of tensor needs to be truncated due to OOM issues
x_train_tensor = x_train_tensor[:,:max_length]
x_train_tensor = x_train_tensor.to_tensor(default_value=0)



In [49]:

#x_train_tensor = tf.slice(x_train_tensor, [0,0], (x_train_tensor.shape[0], max_length))

In [50]:
del train_x

In [51]:
x_test_tensor = en_tokenizer.tokenize(test_x['English'])
x_test_tensor = x_test_tensor.merge_dims(-2,-1)
x_test_tensor = x_test_tensor[:,:max_length]
x_test_tensor = x_test_tensor.to_tensor(default_value=0)
#x_test_tensor = tf.slice(x_test_tensor, [0,0], (x_test_tensor.shape[0], max_length))

In [52]:
del test_x

In [53]:
y_train_tensor = es_tokenizer.tokenize(train_y['Translated'])
y_train_tensor = y_train_tensor.merge_dims(-2,-1)
y_train_tensor = y_train_tensor[:,:max_length]



In [54]:
#0 is the ID for the [PAD] token
y_train_tensor = y_train_tensor.to_tensor(default_value=0)
#y_train_tensor = tf.slice(y_train_tensor, [0,0], (y_train_tensor.shape[0], max_length))

In [55]:
del train_y

In [56]:
y_test_tensor = es_tokenizer.tokenize(test_y['Translated'])
y_test_tensor = y_test_tensor.merge_dims(-2,-1)
y_test_tensor = y_test_tensor[:,:max_length]

In [57]:
y_test_tensor = y_test_tensor.to_tensor(default_value=0)
#y_test_tensor = tf.slice(y_test_tensor, [0,0], (y_test_tensor.shape[0], max_length))

In [58]:
del test_y

In [59]:
print(dir())

['CountVectorizer', 'Dense', 'E', 'Embedding', 'In', 'Input', 'LayerNormalization', 'Model', 'MultiHeadAttention', 'Out', 'Sequential', 'Softmax', 'T', 'TextVectorization', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i10', '_i11', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', '_i18', '_i19', '_i2', '_i20', '_i21', '_i22', '_i23', '_i24', '_i25', '_i26', '_i27', '_i28', '_i29', '_i3', '_i30', '_i31', '_i32', '_i33', '_i34', '_i35', '_i36', '_i37', '_i38', '_i39', '_i4', '_i40', '_i41', '_i42', '_i43', '_i44', '_i45', '_i46', '_i47', '_i48', '_i49', '_i5', '_i50', '_i51', '_i52', '_i53', '_i54', '_i55', '_i56', '_i57', '_i58', '_i59', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'bert_tokenizer_params', 'bert_vocab', 'bert_vocab_org_args', 'bert_vocab_tar_args', 'd', 'en_tokenizer', 'es_tokenizer', 'exit', 'file', 'filepath', 'fp', 'gc', 'get_ipython', 'input_vocab', 'keras', 'keras_n

In [60]:
print(x_train_tensor.shape)
print(x_test_tensor.shape)
print(y_train_tensor.shape)


(None, None)
(None, None)
(None, None)


In [61]:
print(y_test_tensor.shape)

(None, None)


In [62]:
print(x_train_tensor[0])

Tensor("strided_slice:0", shape=(None,), dtype=int64)


In [63]:
#(2917819, 611)
#(2917820, 777)
#(2917819, 622)
#(2917820, 794)

In [64]:
#org_length = 777
#tar_length = 794


In [65]:
#if x_train_tensor.shape[1] < org_length:
#    x_train_tensor = tf.concat([x_train_tensor, tf.zeroes((x_train_tensor[0], (org_length - x_train_tensor.shape[1])))])

In [66]:
#if x_test_tensor.shape[1] < org_length:
#    x_test_tensor = tf.concat([x_test_tensor, tf.zeroes((x_test_tensor[0], (org_length - x_test_tensor.shape[1])))])

In [67]:
#if y_train_tensor.shape[1] < tar_length:
#    y_train_tensor = tf.concat([y_train_tensor, tf.zeroes((y_train_tensor[0], (tar_length - y_train_tensor.shape[1])))])

In [68]:
#if y_test_tensor.shape[1] < tar_length:
#    y_test_tensor = tf.concat([y_test_tensor, tf.zeroes((y_test_tensor[0], (tar_length - y_test_tensor.shape[1])))])

In [69]:
print(x_train_tensor.shape)
print(x_test_tensor.shape)
print(y_train_tensor.shape)
print(y_test_tensor.shape)

(None, None)
(None, None)
(None, None)
(None, None)


In [70]:
#STOP

In [71]:
#Actual Model:
#For now, let's start by creating an exact copy of the attention is all you need model.
#I tried for a long time to make a model using only the Keras Functional API,
#but it seems like I will need to create some custom layers in order to succeed.


In [72]:
def point_wise_feed_forward_network(
  d_model, # Input/output dimensionality.
  dff # Inner-layer dimensionality.
  ):

  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # Shape `(batch_size, seq_len, dff)`.
      tf.keras.layers.Dense(d_model)  # Shape `(batch_size, seq_len, d_model)`.
  ])

In [116]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self,*,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1
               ):
        super(EncoderLayer, self).__init__()


        self.embed_dim = d_model
        self.dense_dim = dff
        self.num_heads = num_attention_heads
        
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        
        self.dense_proj = point_wise_feed_forward_network(d_model, dff)

        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()


        # Dropout for the point-wise feed-forward network.
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
    
        self.supports_masking = True

    def call(self, inputs, mask=None):


        attention_output = self.attention(query=inputs, value=inputs, key=inputs)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [117]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self,
               *,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1
               ):
        super(DecoderLayer, self).__init__()

        # Masked multi-head self-attention.
        self.attention_1 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_attention_heads,
            key_dim=d_model, # Size of each attention head for query Q and key K.
            dropout=dropout_rate
        )
        # Multi-head cross-attention.
        self.attention_2 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_attention_heads,
            key_dim=d_model, # Size of each attention head for query Q and key K.
            dropout=dropout_rate
        )

        # Point-wise feed-forward network.
        self.dense_proj = point_wise_feed_forward_network(d_model, dff)

        # Layer normalization.
        self.layernorm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.add = tf.keras.layers.Add()  # instead of `+` to preserve mask

        # Dropout for the point-wise feed-forward network.
        #self.dropout1 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, inputs, encoder_outputs, mask=None):
        # The encoder output shape is `(batch_size, input_seq_len, d_model)`.

    
        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, use_causal_mask=True
        )
        out_1 = self.layernorm_1(self.add([inputs, attention_output_1]))

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
        )
        out_2 = self.layernorm_2(self.add([out_1, attention_output_2]))

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(self.add([out_2, proj_output]))


In [118]:
#Original hyperparameters
num_layers = 4
embed_dim = 256
latent_dim = 512
num_heads = 8
dropout_rate = 0.1
epochs=50
batch_size = 128
sequence_length = max_length

In [119]:
Dataset_train = tf.data.Dataset.from_tensor_slices((x_train_tensor, y_train_tensor)).batch(batch_size)
Dataset_test = tf.data.Dataset.from_tensor_slices((x_test_tensor, y_test_tensor)).batch(batch_size)

In [120]:
#tf.compat.v1.disable_eager_execution()

In [121]:
print(Dataset_train)
print(Dataset_test)

<BatchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>


In [122]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, num_layers, dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.num_layers = num_layers
        
        self.enc_layers = [
        EncoderLayer(
          d_model=embed_dim,
          num_attention_heads=num_heads,
          dff=dense_dim,
          dropout_rate=dropout_rate)
        for _ in range(num_layers)]
        
        self.supports_masking = True

    def call(self, inputs, mask=None):
        x = inputs
        for i in range(self.num_layers):
            #x = self.enc_layers[i](x, mask)
            x = self.enc_layers[i](x)
        return x

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config






In [123]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


In [124]:
class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, num_layers,  dropout_rate, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.num_layers = num_layers
        
        self.dec_layers = [
        DecoderLayer(
          d_model=embed_dim,
          num_attention_heads=num_heads,
          dff=latent_dim,
          dropout_rate=dropout_rate)
        for _ in range(num_layers)]
        
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        x = inputs
        for i in range(self.num_layers):
            #x = self.dec_layers[i](x, mask)
            x = self.dec_layers[i](x, encoder_outputs)
        return x

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [125]:
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, input_vocab, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads, num_layers, dropout_rate)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, output_vocab, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads, num_layers, dropout_rate)(x, encoded_seq_inputs)
x = tf.keras.layers.Dropout(0.5)(x)
decoder_outputs = tf.keras.layers.Dense(output_vocab, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [126]:
#tf.compat.v1.enable_eager_execution()

In [127]:
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(Dataset_train, epochs=epochs, validation_data=Dataset_test)

InvalidArgumentError: Graph execution error:

Detected at node 'WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets/WordpieceTokenizeWithOffsets' defined at (most recent call last):
    File "C:\Users\nickg\anaconda4\envs\Env39Copy\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\nickg\anaconda4\envs\Env39Copy\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\nickg\anaconda4\envs\Env39Copy\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\nickg\anaconda4\envs\Env39Copy\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\nickg\anaconda4\envs\Env39Copy\lib\site-packages\ipykernel\kernelapp.py", line 736, in start
      self.io_loop.start()
    File "C:\Users\nickg\anaconda4\envs\Env39Copy\lib\site-packages\tornado\platform\asyncio.py", line 195, in star

In [ ]:
#i = Input(shape=(max_length))

In [ ]:
#r = open("textvectorvocap.txt", "r", encoding="utf-8")
#m = r.readline().split("*")

In [ ]:
#TLayer = TextVectorization(output_sequence_length = max_padding_size, vocabulary = m, output_mode = "int")
#inp = TextVectorization(output_sequence_length = max_padding_size)

In [ ]:
#inp = TLayer(i)

In [ ]:
#print(inp)

In [ ]:
#inp.adapt(train_x)

In [ ]:
#dir(inp)

In [ ]:
#print(inp.get_vocabulary())

In [ ]:
#f = open("textvectorvocap.txt", "w", encoding="utf-8")
#f.write("*".join(inp.get_vocabulary()))
#f.close()

In [ ]:
#r = open("textvectorvocap.txt", "r", encoding="utf-8")
#m = r.readline().split("*")

In [ ]:
#print(m == inp.get_vocabulary())

In [ ]:
#print(inp.vocabulary_size())

In [ ]:
#train_x_vec = inp.predict(train_x)
#test_x_vec = inp.predict(test_x)

In [ ]:
#print(i.__call__())

In [ ]:
#em = Embedding(input_dim = max_length, output_dim = 512)(inp)

In [ ]:
#posen = keras_nlp.layers.SinePositionEncoding()(em)

In [ ]:
#mh1 = MultiHeadAttention(num_heads=3, key_dim=512, value_dim=512)

In [ ]:
#mho1 = mh1(posen, posen, posen)

In [ ]:
#no1 = LayerNormalization()
#ou1 = no1(mho1+posen)

In [ ]:
#fe1 = Dense(512)(ou1)
#ou2 = no1(ou1+fe1)

In [ ]:
#lin = Dense(tar_vocab_length)(ou2)

In [ ]:
#s = Softmax()(lin)

In [ ]:
#model = Model(inputs=i, outputs=s)
#print(model.summary())

In [ ]:
#Need to figure out metrics for language translation